In [30]:
import sqlite3 as sql 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
basename = "IceAgeHoehle"

In [32]:
db = f"plankermira/out/{basename}.sql"
CAVENAME = "IceAgeHoehle"
conn = sql.connect(':memory:')
conn.executescript(open(db).read())

In [33]:
def trig_to_cart(length, bearing, gradient):
    n = np.array([np.cos(np.pi/2 - bearing* np.pi / 180), 
              np.sin(np.pi/2 - bearing* np.pi / 180), 
              np.sin(gradient* np.pi / 180)])
    
    v = length * n / np.linalg.norm(n)
    return v

In [34]:
def returnlrud(stn_id, c):

    # collect LRUD data from a single shot

    c.execute(f"SELECT su.NAME, stn1.NAME, stn2.NAME, sh.LENGTH, sh.BEARING, sh.GRADIENT, sf.FLAG \
            from STATION as stn1, STATION as stn2, SHOT as sh, SURVEY as su \
            LEFT OUTER JOIN SHOT_FLAG sf \
            ON sh.ID == sf.SHOT_ID \
            where stn1.SURVEY_ID == su.ID and stn1.ID == {stn_id} and stn1.ID == sh.FROM_ID and stn2.ID == sh.TO_ID;")

    columns = ["su.NAME", "stn1.NAME", "stn2.NAME", "sh.LENGTH", "sh.BEARING", "sh.GRADIENT", "sf.FLAG" ]


    data = pd.DataFrame(c.fetchall(), columns=columns)

    shots = data[data["stn2.NAME"]!="."]
    splays = data[data["stn2.NAME"]=="."]

    normal_bearing, normal_gradient = np.average(shots[["sh.BEARING", "sh.GRADIENT"]], axis =0)

    n_hat = trig_to_cart(1, normal_bearing, normal_gradient)

    # find the two plane vectors. One is horizontal, 

    u = np.array([n_hat[1], -n_hat[0], 0])
    u_hat = u / np.linalg.norm(u)
    v_hat = np.cross(u_hat, n_hat)

    # project the splays in a plane defined by nhat. 

    splays_proj = []
    for splay in splays.values:
        l, bearing, gradient = splay[[3,4,5]]
        v = trig_to_cart(l, bearing, gradient)

        if np.abs(n_hat @ v / np.linalg.norm(v)) < 0.4:
            spl_proj = np.array([u_hat @ v, v_hat @ v])
            splays_proj.append(spl_proj)

    splays_proj = np.array(splays_proj)

    if splays_proj.shape[0] >= 4:
        
        left = "{:.2f}".format(np.abs(np.min(splays_proj[:,0])))
        right = "{:.2f}".format(np.abs(np.max(splays_proj[:,0])))
        up =  "{:.2f}".format(np.abs(np.max(splays_proj[:,1])))
        down = "{:.2f}".format(np.abs(np.min(splays_proj[:,1])))
    else:
        left, right, up, down = "0.00", "0.00", "0.00", "0.00"

    return left, right, up, down


In [35]:
c = conn.cursor()

c.execute("SELECT ct.TITLE, ct.TOPO_DATE, su2.NAME, su1.NAME, stn1.NAME, su2.NAME, su1.NAME, stn2.NAME, sh.ADJ_LENGTH, sh.BEARING, sh.GRADIENT, sf.FLAG, stn1.ID \
          from STATION as stn1, STATION as STN2, SHOT as sh, SURVEY as su1, SURVEY as su2, CENTRELINE as ct\
          LEFT OUTER JOIN SHOT_FLAG sf \
          ON sh.ID == sf.SHOT_ID \
          where stn1.ID == sh.FROM_ID and stn2.ID == sh.TO_ID and stn2.NAME != '-' and stn2.NAME != '.' \
          and stn1.SURVEY_ID == su1.ID and su1.PARENT_ID == su2.ID and sh.CENTRELINE_ID == ct.ID;")

In [36]:
shot_data = pd.DataFrame(c.fetchall())

In [37]:
shot_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Eingang,2024-08-16,iah,1,0,iah,1,1,5.796,326.59,-62.5,None,1
1,Eingang,2024-08-16,iah,1,1,iah,1,2,8.842,351.69,-32.7,None,4
2,Eingang,2024-08-16,iah,1,2,iah,1,3,10.185,34.59,-29.0,None,20
3,Eingang,2024-08-16,iah,1,3,iah,1,4,5.705,345.79,-43.9,None,36
4,Eingang,2024-08-16,iah,1,4,iah,1,5,5.773,325.29,-49.3,None,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Norden,2025-08-15,iah,2,22,iah,2,24,5.462,0.20,11.0,None,738
81,Norden,2025-08-15,iah,2,24,iah,2,25,4.677,329.20,1.0,None,740
82,Norden,2025-08-15,iah,2,25,iah,2,26,6.076,231.20,-5.0,None,741
83,Norden,2025-08-15,iah,2,26,iah,2,27,7.963,229.20,15.0,None,742


In [38]:
_2024_dpl = np.sum(shot_data[np.logical_and(shot_data[3].astype(int)>0, shot_data[11]=="dpl")][8].values)
_2024_total = np.sum(shot_data[shot_data[3].astype(int)>0][8].values)

In [39]:
_2024_total -_2024_dpl
print(_2024_total)
print(_2024_total - _2024_dpl)

490.005
490.005


In [40]:
series_titles= shot_data[0].unique()
series_dates = shot_data[1].unique()

In [41]:
topoteams = []
dates = []
for series_date in series_dates:
    topoteam = ""
    sep = " "
    date = sep.join(series_date.split("-")[::-1])
    monthfirst = date[1] + date[0] + date[2]

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{series_date}';")
    for name, surname in c.fetchall():
        topoteam+= f"{name} {surname}, "
    
    topoteams.append(topoteam[:-2])
    dates.append(monthfirst)


In [42]:
series_titles

array(['Eingang', 'Hauptgang', 'Norden'], dtype=object)

In [43]:
duplicate_shot = "#|L#"
survey_name = "SURVEY NAME: {survey_name}"
survey_date = "SURVEY DATE: {survey date}"
survey_team = "SURVEY TEAM: \n{survey team}"
centreline_header = """DECLINATION:    0.00  FORMAT: DMMDLRUDLDAN  CORRECTIONS:  0.00 0.00 0.00

        FROM           TO   LENGTH  BEARING      INC     LEFT       UP     DOWN    RIGHT   FLAGS  COMMENTS"""

In [44]:
feet = 3.2808399

In [45]:
shot_stub = """{: >12}{: >13}{: >9}{: >9}{: >9}{: >9}{: >9}{: >9}{: >9}  {}\n"""

In [46]:
series_titles

array(['Eingang', 'Hauptgang', 'Norden'], dtype=object)

In [47]:
survey_names = ["IAH1", "IAH2", "IAH3"]

In [48]:
dim = np.loadtxt("plankermira/data/IceAgeHoehle/2-Norden/dims.csv", skiprows =1)

In [49]:
dims = [[], dim]

In [55]:
compass_file = ""

for survey_name, serie, title, thedims in zip(survey_names, series_dates, series_titles, dims):

    topoteam = ""
    sep = " "
    date = serie.split("-")[::-1]
    monthfirst = f"{date[1]} {date[0]} {date[2]}"
    

    c.execute(f"SELECT pp.NAME, pp.SURNAME from PERSON pp, CENTRELINE ct, TOPO tp\
              where tp.PERSON_ID == pp.ID and ct.ID == tp.CENTRELINE_ID and ct.TOPO_DATE == '{serie}';")
    
    for name, surname in c.fetchall():
        if f"{name} {surname}" not in topoteam:
            topoteam+= f"{name} {surname}, "

    centreline_stub = """{cavename}
SURVEY NAME: {survey_name}
SURVEY DATE: {formatted_date}   COMMENT: {formatted_name}
SURVEY TEAM:
{formatted_team}
{header}

{shots}
"""
    # extract the shot data  
    ct = shot_data[shot_data[1] == serie]

    formatted_shots = ""
    counter = 0
    for index, shot in ct.iterrows():
        
        stn_from = shot[2] +"." + shot[3] +"." + shot[4]
        stn_to = shot[5] +"." + shot[6] +"." + shot[7]
        tape = f"{float(shot[8]) * feet:.4f}"
        comp = shot[9] 
        inc = shot[10]
        stn1id = shot[12]

        if len(thedims) !=0:
            print(thedims, type(thedims))
            l, r, u, d = thedims[counter]*feet
        else: 
            l, r, u, d = 0.00, 0.00, 0.00, 0.00

        if shot[11] == "dpl":
            flag = duplicate_shot
        else:
            flag = " "

        formatted_shots+= shot_stub.format(stn_from, 
                                           stn_to, 
                                           tape, 
                                           comp, 
                                           inc,
                                           f"{l:.2f}", 
                                           f"{u:.2f}", 
                                           f"{d:.2f}", 
                                           f"{r:.2f}", 
                                           flag)

        counter+=1
    formatted_centreline = centreline_stub.format(cavename =CAVENAME,
                                                  survey_name = survey_name,
                                                  formatted_name = title, 
                                                  formatted_date = monthfirst,
                                                  formatted_team = topoteam[:-2],
                                                  header = centreline_header,
                                                  shots = formatted_shots)
    
    compass_file += formatted_centreline

[[3.8  3.1  3.3  1.8 ]
 [4.   3.   1.5  1.  ]
 [2.5  2.   0.5  0.5 ]
 [0.   1.   1.   0.5 ]
 [1.8  1.5  1.   0.  ]
 [1.   0.8  1.   0.8 ]
 [0.5  1.5  0.4  0.15]
 [1.7  3.   0.4  1.  ]
 [7.   4.7  1.2  2.4 ]
 [1.7  0.   2.   0.5 ]
 [2.   2.   5.   1.  ]
 [3.7  4.   3.   1.6 ]
 [0.1  1.8  0.4  0.5 ]
 [0.3  1.   1.   1.5 ]
 [2.2  2.2  2.   1.5 ]
 [4.2  2.9  2.2  1.1 ]
 [1.5  3.   1.2  4.  ]
 [1.   5.   7.   0.  ]
 [0.1  0.2  0.4  0.4 ]
 [0.   1.4  4.   1.9 ]
 [0.   3.   5.5  2.  ]
 [1.   5.   3.5  0.  ]
 [0.   0.   0.   0.  ]
 [1.5  0.   0.4  1.  ]
 [0.   0.   1.   1.  ]
 [0.   0.   1.   1.  ]
 [2.5  1.5  1.   9.  ]
 [2.5  2.5  1.   9.  ]] <class 'numpy.ndarray'>
[[3.8  3.1  3.3  1.8 ]
 [4.   3.   1.5  1.  ]
 [2.5  2.   0.5  0.5 ]
 [0.   1.   1.   0.5 ]
 [1.8  1.5  1.   0.  ]
 [1.   0.8  1.   0.8 ]
 [0.5  1.5  0.4  0.15]
 [1.7  3.   0.4  1.  ]
 [7.   4.7  1.2  2.4 ]
 [1.7  0.   2.   0.5 ]
 [2.   2.   5.   1.  ]
 [3.7  4.   3.   1.6 ]
 [0.1  1.8  0.4  0.5 ]
 [0.3  1.   1.   1.5 ]
 [2.2  2.

In [56]:
with open("plankermira/out/IceAgeHoehle_2025.DAT", "w") as f:
    f.write(compass_file)
    f.close()

In [20]:
38.21 * feet

125.36089257900001